### Tests
Can the LLM write decent unit tests for our functions?

There's only two functions and Ive pasted them into a py file for easy access. Lets import them.

In [2]:
from project_functions import objective, app

I'll now paste the functions into LLM and ask it to create unit tests for them.

**Prompt:** use pytest to test this function

**Follow up Prompt:** The function is an objective for the Optuna library to optimize. It does not have any inputs. It creates and xgboost model and then returns the accuracy of the model. The function is fed into Optuna's study.optimize().

**Result:** 

In [4]:
import pytest
import unittest.mock as mock
import optuna
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

def test_objective():
   # Load a sample dataset
   iris = load_iris()
   X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3, random_state=1)

   # Mock the trial object
   trial = optuna.trial.Trial(
       number=0,
       state=optuna.trial.TrialState.RUNNING,
       value=None,
       datetime_start=None,
       datetime_complete=None,
       params={},
       distributions={},
       user_attrs={},
       system_attrs={},
       intermediate_values={},
       trial_id=0,
       study_id=0,
       number=0
   )

   # Define fixed hyperparameters
   trial.suggest_int = lambda name, low, high: 100 # n_estimators
   trial.suggest_int = lambda name, low, high: 1 # max_depth
   trial.suggest_float = lambda name, low, high, log: 0.5 # min_child_weight, learning_rate, gamma, reg_alpha, reg_lambda
   trial.suggest_float = lambda name, low, high: 0.5 # subsample, colsample_bytree

   # Call the function
   accuracy = objective(trial)

   # Check the accuracy
   xgb_model = XGBClassifier(n_estimators=100, max_depth=1, min_child_weight=0.5, learning_rate=0.5, gamma=0.5, subsample=0.5, colsample_bytree=0.5, reg_alpha=0.5, reg_lambda=0.5, random_state=99)
   xgb_model.fit(X_train, y_train)
   y_pred = xgb_model.predict(X_test)
   expected_accuracy = accuracy_score(y_test, y_pred)
   assert accuracy == expected_accuracy


SyntaxError: keyword argument repeated: number (1556878068.py, line 28)

Oh dear, thats no good. Many things are wrong here.

it imported unittest.mock and then never used it (the commentary said it had to mock the data).

it did not need to actually import pytest.

there is a repeated keyword argument.

the keyword arguments are all completely wrong and dont even exist in that class.

its trying to create fixed hyper parameters but isnt useing the class optuna.trial.FixedTrial.

it then tries to compare the objective accuracy to a newly fitted models accuracy. They will certainly be different since the former is based on cross validation and the latter is not.

I'd say, this is unsalvageable. I tried telling the model that the trial object took different keywords. It understood and revised the test but it was still nonesense.

How about our FastAPI app?

**Prompt:** I have a function that uses pydantic's Basemodel and FastAPI to return a prediction for an xgboost model. write a test for this functions using py test.

**Follow up Prompt:** The data will be in json format. The Data model consistes of our ml model's features and dtypes. The xgboost model is a XGBClassifier object that is already in memory (its just called "model")

**Result:** 

In [1]:
import pickle
with open('xgb_model.pkl', 'rb') as file:
   model = pickle.load(file)

In [4]:
from fastapi.testclient import TestClient

ModuleNotFoundError: No module named 'httpx'

In [ ]:
from fastapi.testclient import TestClient
#from main import app, model # assuming your FastAPI app and model are defined in a file named main.py

client = TestClient(app)

def test_predict():
   # Define test data
   test_data = {
       "tenure": 12,
       "MonthlyCharges": 20,
       "TotalCharges": 240,
       "TotalServices": 3,
       "gender_Male": 1,
       "InternetService_Fiber_optic": 0,
       "InternetService_No": 1,
       "Contract_One_year": 0,
       "Contract_Two_year": 1,
       "PaymentMethod_Credit_card_automatic": 0,
       "PaymentMethod_Electronic_check": 1,
       "PaymentMethod_Mailed_check": 0,
       "SeniorCitizen": 0,
       "Partner": 1,
       "Dependents": 0,
       "PhoneService": 1,
       "MultipleLines": 0,
       "OnlineSecurity": 0,
       "OnlineBackup": 0,
       "DeviceProtection": 1,
       "TechSupport": 0,
       "StreamingTV": 0,
       "StreamingMovies": 0,
       "PaperlessBilling": 1,
   }

   # Send a POST request to the /predict endpoint
   response = client.post("/predict", json=test_data)

   # Check the status code
   assert response.status_code == 200

   # Check the returned prediction
   assert "prediction" in response.json()

Alright. This seems to align with the official documentation: https://fastapi.tiangolo.com/tutorial/testing/

it got the variable names partially wrong though (remember that i used aliases in my app). easy to change.

I suppose this is satisfactory. There could be other tests which check the actual returned values. Though this wouldnt sctrictly be testing app functionality which is what we had in mind with our prompts.

**Conclusion**

Half marks for the LLM here. Took care of FastAPI app no problem but was fairly hopeless at the Optuna objective function